In [30]:
%pip install gradio
import gradio as gr
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pandas as pd
import pickle


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
# TODO change the file to your own model.
model_filename = "random_forest_regression.pkl"

random_forest_model = RandomForestRegressor()
with open(model_filename, 'rb') as f:
    random_forest_model = pickle.load(f)

print('Number of features: ', random_forest_model.n_features_in_)
print('Features are (see week 1): ', ['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income'])
random_forest_model

Number of features:  7
Features are (see week 1):  ['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income']


/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


RandomForestRegressor(random_state=42)

In [32]:
df_bfs_data = pd.read_csv('apartments_data_enriched_lat_lon_combined.csv', sep=',', encoding='utf-8')
## df_bfs_data['tax_income'] = df_bfs_data['tax_income'].str.replace("'", "").astype(float)

## Additional feature

In [33]:
# install geopy
%pip install geopy

from geopy.distance import geodesic
import pandas as pd

# Liste der Bahnhöfe im Kanton Zürich
train_stations = {
    "Zürich HB": (47.378177, 8.540192),
    "Zürich Oerlikon": (47.4116, 8.5446),
    "Zürich Altstetten": (47.3913, 8.4850),
    "Zürich Enge": (47.3643, 8.5312),
    "Zürich Stadelhofen": (47.3663, 8.5485),
    "Winterthur": (47.4998, 8.7257),
    "Dietikon": (47.4052, 8.4009),
    "Uster": (47.3476, 8.7207),
    "Dübendorf": (47.3978, 8.6189),
    "Wetzikon": (47.3275, 8.7976),
    "Bülach": (47.5202, 8.5385),
    "Meilen": (47.2699, 8.6458),
    "Thalwil": (47.2911, 8.5647),
    "Horgen": (47.2597, 8.5975)
}

# Funktion zur Berechnung der Distanz zum nächsten Bahnhof
def get_nearest_station_distance(lat, lon):
    return min([geodesic((lat, lon), coords).km for coords in train_stations.values()])

# Falls dein DataFrame bereits Latitude/Longitude-Spalten hat:
df_bfs_data['distance_to_train_station'] = df_bfs_data.apply(lambda row: get_nearest_station_distance(row['lat'], row['lon']), axis=1)

# Optional: Name des nächstgelegenen Bahnhofs speichern
def get_nearest_station_name(lat, lon):
    return min(train_stations.keys(), key=lambda station: geodesic((lat, lon), train_stations[station]).km)

df_bfs_data['nearest_train_station'] = df_bfs_data.apply(lambda row: get_nearest_station_name(row['lat'], row['lon']), axis=1)

# Ergebnis anzeigen
df_bfs_data[['distance_to_train_station', 'nearest_train_station']].head()


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


,distance_to_train_station,nearest_train_station
0,0.158395,Zürich Oerlikon
1,0.462351,Zürich Oerlikon
2,1.039853,Zürich Oerlikon
3,0.258770,Zürich Oerlikon
4,0.611426,Zürich Oerlikon


### Show first 5 rows

In [34]:
# Show the last 5 rows of the data
df_bfs_data.tail()

,bfs_number,rooms,area,price,postalcode,address,town,description_raw,bfs_name,pop,pop_dens,frg_pct,emp,tax_income,lat,lon,x,y,distance_to_train_station,nearest_train_station
2395,3,4.5,104,2055,8906,"Dorfstr. 17, 8906 Bonstetten",Bonstetten,«Grosszügige 4.5 Maisonette WHG sucht Nachmieter»,Bonstetten,5572,749.932705,16.564968,1014.0,91002,47.314835,8.469243,240951.375000,677922.0625,7.222698,Zürich Enge
2396,196,4.5,98,1950,8617,"Langenmattstrasse 14, 8617 Mönchaltorf",Mönchaltorf,«Naturnahes Familienparadies: 4.5-Zimmer-Wohnu...,Mönchaltorf,3898,513.570487,17.162648,1360.0,78986,47.315971,8.719728,241357.031250,696857.1250,3.517152,Uster
2397,4,5.5,175,3720,8915,"Oberalbis 10, 8915 Hausen am Albis",Hausen am Albis,«Wohnung inGrosser stimmungsvoller 5 1/2 Haust...,Hausen am Albis,3751,275.808824,16.022394,1021.0,91766,47.252140,8.530311,234044.109375,682635.9375,5.052998,Thalwil
2398,173,4.5,130,2400,8335,"Isikerstrasse 26, 8335 Hittnau",Hittnau,«Gemütliches Wohnen in Hittnau»,Hittnau,3664,282.934363,7.669214,783.0,79838,47.370026,8.819698,247494.140625,704308.8750,5.014088,Wetzikon
2399,192,3.5,102,2272,8132,"Bachtelweg 2, 8132 Egg b. Zürich",Egg b. Zürich,«Terrassen Wohnung mit schöner Aussicht zu ver...,Egg,8796,605.368204,19.918145,2469.0,94471,47.297756,8.692446,239298.828125,694827.0000,4.695098,Meilen


## Feature definition

## Model performance

In [35]:
def model_performance(features, df_bfs_data, random_forest_model=RandomForestRegressor(random_state=42)):
    df_bfs_data = df_bfs_data.sample(frac=1, random_state=42)
    X, y = df_bfs_data[features], df_bfs_data['price']
    scores = cross_val_score(random_forest_model, X, y, scoring="neg_root_mean_squared_error", cv=5)
    print('CV results RMSE: ', np.round(scores))
    print('Mean RMSE: ', np.mean(np.round(scores, 0)))
    return scores

In [ ]:
def predict_price(model, rooms, area, pop, pop_dens, frg_pct, emp, tax_income, distance_to_train_station):
    input_data = np.array([[rooms, area, pop, pop_dens, frg_pct, emp, tax_income, distance_to_train_station]])
    return model.predict(input_data)[0]

In [37]:
old_features = ['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income']
new_features = old_features + ['distance_to_train_station']
print("Performance with original features:")
old_scores = model_performance(old_features, df_bfs_data)

# Evaluate performance with new features including distance to train station
print("\nPerformance with added train station distance feature:")
new_scores = model_performance(new_features, df_bfs_data)

# Train the model with the new features
final_model = RandomForestRegressor(random_state=42)
X = df_bfs_data[new_features]
y = df_bfs_data['price']
final_model.fit(X, y)


Performance with original features:
CV results RMSE:  [-837. -687. -749. -891. -629.]
Mean RMSE:  -758.6

Performance with added train station distance feature:
CV results RMSE:  [-781. -600. -682. -838. -589.]
Mean RMSE:  -698.0


RandomForestRegressor(random_state=42)

In [38]:
locations = {
    "Zürich": 261,
    "Kloten": 62,
    "Uster": 198,
    "Illnau-Effretikon": 296,
    "Feuerthalen": 27,
    "Pfäffikon": 177,
    "Ottenbach": 11,
    "Dübendorf": 191,
    "Richterswil": 138,
    "Maur": 195,
    "Embrach": 56,
    "Bülach": 53,
    "Winterthur": 230,
    "Oetwil am See": 157,
    "Russikon": 178,
    "Obfelden": 10,
    "Wald (ZH)": 120,
    "Niederweningen": 91,
    "Dällikon": 84,
    "Buchs (ZH)": 83,
    "Rüti (ZH)": 118,
    "Hittnau": 173,
    "Bassersdorf": 52,
    "Glattfelden": 58,
    "Opfikon": 66,
    "Hinwil": 117,
    "Regensberg": 95,
    "Langnau am Albis": 136,
    "Dietikon": 243,
    "Erlenbach (ZH)": 151,
    "Kappel am Albis": 6,
    "Stäfa": 158,
    "Zell (ZH)": 231,
    "Turbenthal": 228,
    "Oberglatt": 92,
    "Winkel": 72,
    "Volketswil": 199,
    "Kilchberg (ZH)": 135,
    "Wetzikon (ZH)": 121,
    "Zumikon": 160,
    "Weisslingen": 180,
    "Elsau": 219,
    "Hettlingen": 221,
    "Rüschlikon": 139,
    "Stallikon": 13,
    "Dielsdorf": 86,
    "Wallisellen": 69,
    "Dietlikon": 54,
    "Meilen": 156,
    "Wangen-Brüttisellen": 200,
    "Flaach": 28,
    "Regensdorf": 96,
    "Niederhasli": 90,
    "Bauma": 297,
    "Aesch (ZH)": 241,
    "Schlieren": 247,
    "Dürnten": 113,
    "Unterengstringen": 249,
    "Gossau (ZH)": 115,
    "Oberengstringen": 245,
    "Schleinikon": 98,
    "Aeugst am Albis": 1,
    "Rheinau": 38,
    "Höri": 60,
    "Rickenbach (ZH)": 225,
    "Rafz": 67,
    "Adliswil": 131,
    "Zollikon": 161,
    "Urdorf": 250,
    "Hombrechtikon": 153,
    "Birmensdorf (ZH)": 242,
    "Fehraltorf": 172,
    "Weiach": 102,
    "Männedorf": 155,
    "Küsnacht (ZH)": 154,
    "Hausen am Albis": 4,
    "Hochfelden": 59,
    "Fällanden": 193,
    "Greifensee": 194,
    "Mönchaltorf": 196,
    "Dägerlen": 214,
    "Thalheim an der Thur": 39,
    "Uetikon am See": 159,
    "Seuzach": 227,
    "Uitikon": 248,
    "Affoltern am Albis": 2,
    "Geroldswil": 244,
    "Niederglatt": 89,
    "Thalwil": 141,
    "Rorbas": 68,
    "Pfungen": 224,
    "Weiningen (ZH)": 251,
    "Bubikon": 112,
    "Neftenbach": 223,
    "Mettmenstetten": 9,
    "Otelfingen": 94,
    "Flurlingen": 29,
    "Stadel": 100,
    "Grüningen": 116,
    "Henggart": 31,
    "Dachsen": 25,
    "Bonstetten": 3,
    "Bachenbülach": 51,
    "Horgen": 295
}

In [39]:
# Define the core prediction function
def predict_apartment(rooms, area, town):
    bfs_number = locations[town]
    df = df_bfs_data[df_bfs_data['bfs_number']==bfs_number].copy()
    df.reset_index(inplace=True)
    df.loc[0, 'rooms'] = rooms
    df.loc[0, 'area'] = area
    if len(df) != 1: # if there are more than two records with the same bfs_number reutrn -1
        return -1
    prediction = random_forest_model.predict(df[['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income']])
    return np.round(prediction[0], 0)

In [40]:
predict_apartment(3, 100, 'Zürich')

-1

In [41]:
# Create the Gradio interface
iface = gr.Interface(
    fn=predict_apartment,
    inputs=["number", "number", gr.Dropdown(choices=locations.keys(), label="Town", type="value")],
    outputs=[gr.Number()],
    examples=[[4.5, 120, "Dietlikon"], [3.5, 60, "Winterthur"]]
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
